<a href="https://colab.research.google.com/github/rotogni/diffusion-lagr/blob/master/diff_lagr_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set up environment for guided diffusion project
!apt-get update
!apt-get install -y libopenmpi-dev openmpi-bin
!pip install mpi4py
!pip install --no-binary=h5py h5py

# Clone the repository (replace with the actual repo URL)
!git clone https://github.com/rotogni/diffusion-lagr.git
%cd diffusion-lagr

# Install the package in development mode
!pip install -e .

# Verify installation
!python -c "import guided_diffusion; print('guided_diffusion installed successfully')"

In [ ]:
# Set the flags
DATA_FLAGS="--dataset_path datasets/trajectories.h5 --dataset_name train"
MODEL_FLAGS="--dims 1 --image_size 295 --in_channels 3 --num_channels 128 --num_res_blocks 3 --attention_resolutions 250,125 --channel_mult 1,1,2,3,4"
DIFFUSION_FLAGS="--diffusion_steps 800 --noise_schedule tanh6,1"
TRAIN_FLAGS="--lr 1e-4 --batch_size 64"

In [ ]:
# Training command
!python scripts/turb_train.py $DATA_FLAGS $MODEL_FLAGS $DIFFUSION_FLAGS $TRAIN_FLAGS

In [ ]:
# Set sampling flags
SAMPLE_FLAGS="--num_samples 200 --batch_size 64 --model_path ema_0.9999_250000.pt"

#Please note that the $MODEL_FLAGS and $DIFFUSION_FLAGS should be the same as those used in training.

In [ ]:
# Sampling command
!python scripts/turb_sample.py $SAMPLE_FLAGS $MODEL_FLAGS $DIFFUSION_FLAGS


In [ ]:
# After sampling with the above command, it will generate a file named samples_179200x2000x3.npz (for DM-3c as an example).
# You can use the following code to read and retrieve the generated velocities:

import h5py
import numpy as np

with h5py.File('datasets/trajectories.h5', 'r') as h5f:
    rx0 = np.array(h5f.get('min'))
    rx1 = np.array(h5f.get('max'))

u3c = (np.load('samples_179200x2000x3.npz')['arr_0']+1)*(rx1-rx0)/2 + rx0
#Just like for training, you can use multiple GPUs for sampling.
